# ML model for Keyword Classification - Tech Notebook
This notebook introduces (1) how to explore, prepare and preprocess the datasets; (2) how to train and evaluate the ML model; and (3) how to use this trained ML model, for technical audiences.
## Problem Description
The AODN catalogue $C=\{M, K, P\}$ serves as a platform for storing datasets and their associated metadata. $M=\{m_1,m_2,\ldots, m_x\}$ is a set of metadata records which are used to describe the dataset in AODN catalogue $C$. $K=\{k_1, k_2, \ldots, k_y\}$ is a set of pre-defined keywords that are used to categorise dataset. In the catalogue $C = \{M, K\}$, a subset of metadata records, $M_t \subseteq M$, have not yet been categorised with keywords. For these records, $K_i = \emptyset $ for all $m_i \in M_t$. Given another subset of metadata records, $M_s \subseteq M$, where each record has already been categorised with keywords (i.e., $K_i \neq \emptyset $ for all $m_i \in M_s$). The research question is as follows:

How to design and develop a machine learning model, denoted as $MM_{keywords}$, that can automatically label the uncategorised metadata records $M_t$ using a predefined set of keywords $K$. Specifically, the model should be trained to learn a mapping rule $d_i \mapsto K_i$ based on the observed patterns from the sample set $M_s$, where each description $d_i$ of a metadata record $m_i \in M_s$ is associated with a set of keywords $K_i$. Once trained, the model should be able to apply this learned mapping to accurately categorise the records in $M_t$ by assigning their corresponding keywords based on the records' descriptions.

To simplify the task, we restrict the scope of keywords to those falling within the primary AODN vocabulary:
- AODN Instrument Vocabulary
- AODN Discovery Parameter Vocabulary
- AODN Platform Vocabulary

Only keywords $k_j \in K_i$ that are part of the listed AODN vocabularies will be considered. Any keyword not belonging to these vocabularies will be excluded from $K_i$ for all metadata records in the categorised metadata set $M_s$.

### Formal Definitions
- **Definition 1: A metadata record $m_i=(d_i, K_i), m_i \in M$** is a record describing a dataset. Specifically, $i$ is the unique identifier of the record. $d_i$ is a textual abstract that serves as the description of the dataset. $K_i \subseteq K$ is a subset of keywords used to label the dataset.
- **Definition 2: A abstract $d_i$** is a piece of textual information which is used to describe the dataset. The embedding $\mathbf{d_i}$ is a vector representation of the textual description $d_i$, calculated using the "bert-base-uncased" model. The embedding vector $\mathbf{d_i}$ for each abstract $d_i$ has an universal dimensionality, denoted as $dim=|\mathbf{d_i}|$. A feature matrix $\mathbf{X}$ of a shape $|M_s| \times dim$ aggregates the embeddings for the abstacts of all samples in $M_s$, where |M_s is the total number of metadata records.
- **Definition 3: A keyword $k_j$** is a predefined label used for catogarising datasets. Each metadata record $m_i$ is associated with a set of keywords $K_i \subseteq K$, while $K$ is the complete set of predefined keywords. The keywords $K_i$ for a metadata record $m_i$ is mathematiacally represented as a binary vector $y_i$ with a size of $|K|$. where each element indicates the presence or absence of a specific label. A value of 1 at position $j$ denotes the label $k_j \in K$ is present in the metadata record $m_i$, in this sence $k_j \in K_i$, while a value of 0 indicates its absence. A target matrix $\mathbf{Y}$ is a $|M_s| \times |K|$ binary matrix, where $|M_s|$ is the size of the metadata records set $M_s=\{m_1,m_2,\ldots, m_x\}$, and $|K|$ is the size of the keywords set $K=\{k_1, k_2, \ldots, k_y\}$. Each entry $ \mathbf{K}[i, j] $ is 1 if metadata record $ m_i $ is associated with keyword $ k_j $, and 0 otherwise.



In [1]:
# add module path for notebook to use
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\data_discovery_ai\\utils")
    sys.path.append(module_path+"\\data_discovery_ai\\model")
    sys.path.append(module_path+"\\data_discovery_ai\\common")

current_path = os.getcwd()

# import modules
import preprocessor
import keywordModel
import constants
import es_connector

c:\Users\yhu12\AppData\Local\miniforge3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As shown in the [framework](data-discovery-ai-framework.drawio.png), three distinct but connected modules work cooperatively as the keyword classifier pipeline. This notebook will go through the functions in these modules to show how we preprocess data, train the ML model, and make predictions.
## Data Preprocessing
The data preprocessing module is used to prepare data for training and testing models. Key features include: getting raw data, preparing sample data, converting textual data to numeric representations, resampling, and preparing input and output matrices.
### Getting Raw Data
Raw data means the all metadata records $M$ stored in Elasticsearch. A elasticsearch configuration file `esManager.ini` is needed to be created in folder `data_discoverty_ai/common`, in which two fields are required: `end_point` and `api_key`. For more information, please refer to [README](../README.md#file-structure). We first fetch raw data from Elasticsearch.

In [15]:
# load Elasticsearch configuration
import configparser
from pathlib import Path

def load_es_config() -> configparser.ConfigParser:
    elasticsearch_config_file_path = f"../data_discovery_ai/common/{constants.ELASTICSEARCH_CONFIG}"
    esConfig = configparser.ConfigParser()
    esConfig.read(elasticsearch_config_file_path)
    return esConfig

In [16]:
# connect and query Elasticsearch
esConfig = load_es_config()
client = es_connector.connect_es(esConfig)
raw_data = es_connector.search_es(client)

searching elasticsearch: 100%|██████████| 100/100 [02:05<00:00,  1.25s/it]


### Identify Samples
Sample set is a subset of the raw dataset. A sample set $M_s$ is a set of metadata records in which keywords contain particular AODN vocabus. We first identify samples from raw data, and then preprocess the sample set.

In [32]:
# get predefined vocabs
def load_keyword_config() -> configparser.ConfigParser:
    keyword_config_file_path = f"../data_discovery_ai/common/{constants.KEYWORD_CONFIG}"
    keywordConfig = configparser.ConfigParser()
    keywordConfig.read(keyword_config_file_path)
    return keywordConfig
keywordConfig = load_keyword_config()
vocabs = keywordConfig["preprocessor"]["vocabs"].split(", ")
vocabs

['AODN Instrument Vocabulary',
 'AODN Discovery Parameter Vocabulary',
 'AODN Platform Vocabulary']

In [34]:
# identify samples with predefined vocabs
labelledDS = preprocessor.identify_sample(raw_data, vocabs)
labelledDS.iloc[0]["keywords"]

[{'concepts': [{'id': 'Practical salinity of the water body',
    'url': 'http://vocab.nerc.ac.uk/collection/P01/current/PSLTZZ01'},
   {'id': 'Temperature of the water body',
    'url': 'http://vocab.nerc.ac.uk/collection/P01/current/TEMPPR01'},
   {'id': 'Fluorescence of the water body',
    'url': 'http://vocab.nerc.ac.uk/collection/P01/current/FLUOZZZZ'},
   {'id': 'Turbidity of the water body',
    'url': 'http://vocab.nerc.ac.uk/collection/P01/current/TURBXXXX'}],
  'scheme': 'theme',
  'description': '',
  'title': 'AODN Discovery Parameter Vocabulary'}]

The keywords is in a nested json format, we need to flattern them.

In [37]:
preprocessed_samples = preprocessor.sample_preprocessor(labelledDS, vocabs)
preprocessed_samples.describe()

KeyError: 'concepts'